In [1]:
import nibabel as nib
import os
from augmentation import *
import numpy as np
import random
from tqdm import tqdm
from pathlib import Path
import csv
import pandas as pd
import joblib
from skimage import measure
import pandas as pd
from scipy.spatial import ConvexHull
from scipy.stats import skew, kurtosis
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.feature import structure_tensor, structure_tensor_eigenvalues
from skimage.feature import hessian_matrix, hessian_matrix_eigvals


mod1 = 'DWI_800.nii.gz'
mod2 = 'GED1.nii.gz'
mod3 = 'GED2.nii.gz'
mod4 = 'GED3.nii.gz'
mod5 = 'GED4.nii.gz'
mod6 = 'T1.nii.gz'
mod7 = 'T2.nii.gz'
mod8 = 'mask_GED4.nii.gz'

mods = {'DWI': mod1, 'GED1': mod2, 'GED2': mod3, 'GED3': mod4, 'GED4': mod5, 'T1': mod6, 'T2': mod7, 'mask_GED4': mod8}

vendor_to_number = {'A': 0, 'B1':1, 'B2':2, 'C':3}

In [ ]:
### Match img with mask
# Define the path to the main folder
search_dir_data = Path('C://Users//SCoulY//Downloads//Data//val_data')

# Recursively find all files ending with '.nii.gz'
# The rglob('*.nii.gz') method returns a generator
mod_list = []
for mod in mods.values():
    file_list = list(search_dir_data.rglob(f'*{os.sep}{mod}'))
    mod_list.append(file_list)
    print(f"Mod {mod} found {len(file_list)} files.")

mask_list = []
search_dir_mask = Path('C://Users//SCoulY//Downloads//train_val_all_data_all_modalities//val_all_modalities//')
for mask in search_dir_mask.rglob('*.nii.gz'):
    mask_list.append(mask)
print(f"Found {len(mask_list)} segmentation masks.")

# pair the samples with segmentation masks
target_mods = 'T1' ### Change this to the desired modality
mask_str_list = [str(mask.parent).split(os.sep)[-1] + os.sep + mask.name for mask in mask_list]
mod_roi_list = []

mod_index = list(mods.keys()).index(target_mods)
for sample_GED4 in mod_list[list(mods.keys()).index('GED4')]:  # Use GED4 as the base for matching
    sample = str(sample_GED4).replace('GED4.nii.gz', mods[target_mods])  # Replace 'GED4' with the target modality
    #wrap in Path to ensure compatibility
    sample = Path(sample)
    mod_name = sample.name
    sample_name = str(sample.parent).split(os.sep)[-1]  # Get the last part of the path as sample name
    # print(f"Processing sample: {sample_name} with mod: {mod_name}")
    mask_name = sample_name + os.sep + mods[target_mods].replace('.nii.gz', '_pred.nii.gz') 

    if mask_name in mask_str_list:
        idx = mask_str_list.index(mask_name)
        
        mask = nib.load(mask_list[idx]).get_fdata().astype(np.float32)
        img = nib.load(str(sample).replace(mod_name, f'{target_mods}.nii.gz')).get_fdata().astype(np.float32)
        roi = img[mask>0]
        if roi.size == 0:
            print(f"No ROI found for sample: {sample_name} with mod: {mod_name}")
            continue
        mod_roi_list.append((img, mask, roi, sample_name, target_mods))
    else:
        print(f"No mask found for sample: {sample_name} with mod: {mod_name}")
        #use the GED4 mask as a fallback
        ged4_mask_name = sample_name + os.sep + mods['GED4'].replace('.nii.gz', '_pred.nii.gz') 
        if ged4_mask_name in mask_str_list:
            idx = mask_str_list.index(ged4_mask_name)
            mask = nib.load(mask_list[idx]).get_fdata().astype(np.float32)
            img = nib.load(str(sample).replace(mod_name, 'GED4.nii.gz')).get_fdata().astype(np.float32)
            roi = img[mask>0]
            if roi.size == 0:
                print(f"No ROI found for sample: {sample_name} with mod: {mod_name} using GED4 mask")
                continue
            mod_roi_list.append((img, mask, roi, sample_name, 'GED4'))
        else:
            raise FileNotFoundError(f"No mask found for sample: {sample_name} with mod: {mod_name} and no GED4 mask available.")


Mod DWI_800.nii.gz found 53 files.
Mod GED1.nii.gz found 55 files.
Mod GED2.nii.gz found 54 files.
Mod GED3.nii.gz found 54 files.
Mod GED4.nii.gz found 60 files.
Mod T1.nii.gz found 53 files.
Mod T2.nii.gz found 52 files.
Mod mask_GED4.nii.gz found 0 files.
Found 382 segmentation masks.
C:\Users\SCoulY\Downloads\train_val_all_data_all_modalities\val_all_modalities\0930-A\T1_pred.nii.gz C:\Users\SCoulY\Downloads\Data\val_data\Vendor_A\0930-A\T1.nii.gz
C:\Users\SCoulY\Downloads\train_val_all_data_all_modalities\val_all_modalities\0978-A\T1_pred.nii.gz C:\Users\SCoulY\Downloads\Data\val_data\Vendor_A\0978-A\T1.nii.gz
C:\Users\SCoulY\Downloads\train_val_all_data_all_modalities\val_all_modalities\0985-A\T1_pred.nii.gz C:\Users\SCoulY\Downloads\Data\val_data\Vendor_A\0985-A\T1.nii.gz
C:\Users\SCoulY\Downloads\train_val_all_data_all_modalities\val_all_modalities\1002-A\T1_pred.nii.gz C:\Users\SCoulY\Downloads\Data\val_data\Vendor_A\1002-A\T1.nii.gz
C:\Users\SCoulY\Downloads\train_val_all_dat

In [4]:
### Essential functions for feature extraction

def compute_surface_area(binary_mask):
    """Compute surface area of a 3D binary mask using marching cubes."""
    verts, faces, _, _ = measure.marching_cubes(binary_mask, level=0)
    area = 0.0
    for tri in faces:
        p0, p1, p2 = verts[tri]
        tri_area = 0.5 * np.linalg.norm(np.cross(p1 - p0, p2 - p0))
        area += tri_area
    return area, verts

def compute_convex_volume(verts):
    """Compute convex hull volume from vertices."""
    try:
        hull = ConvexHull(verts)
        return hull.volume
    except:
        return np.nan

def compute_directional_glcm_features(img_3d, mask_3d, distances=[1], levels=256):
    """Compute simple directional GLCM contrast on three orthogonal slices through ROI."""
    # Take central slices through the ROI
    coords = np.array(np.where(mask_3d))
    zc, yc, xc = [int(np.mean(c)) for c in coords]

    features = {}

    # Helper to compute contrast per angle in 2D
    def glcm_contrast(slice2d):
        # quantize
        img = slice2d.astype(np.uint8)
        glcm = graycomatrix(img, distances=distances, angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                            levels=levels, symmetric=True, normed=True)
        contrast = [graycoprops(glcm, 'contrast')[0, i] for i in range(4)]
        return contrast

    # XY slice
    contrast_xy = glcm_contrast(img_3d[zc])
    features['glcm_xy_contrast_mean'] = np.mean(contrast_xy)
    features['glcm_xy_anisotropy'] = (max(contrast_xy) - min(contrast_xy)) / (np.mean(contrast_xy)+1e-8)

    # XZ slice
    contrast_xz = glcm_contrast(img_3d[:, yc, :])
    features['glcm_xz_contrast_mean'] = np.mean(contrast_xz)
    features['glcm_xz_anisotropy'] = (max(contrast_xz) - min(contrast_xz)) / (np.mean(contrast_xz)+1e-8)

    # YZ slice
    contrast_yz = glcm_contrast(img_3d[:, :, xc])
    features['glcm_yz_contrast_mean'] = np.mean(contrast_yz)
    features['glcm_yz_anisotropy'] = (max(contrast_yz) - min(contrast_yz)) / (np.mean(contrast_yz)+1e-8)

    return features

def extract_features_with_intensity_gradient(mask_3d, img_3d):
    mask_3d = mask_3d.astype(bool)
    props = measure.regionprops(mask_3d.astype(np.uint8), intensity_image=img_3d)

    # precompute gradients
    gx, gy, gz = np.gradient(img_3d.astype(np.float32))
    grad_mag = np.sqrt(gx**2 + gy**2 + gz**2)

    # structure tensor
    A_elems = structure_tensor(img_3d, sigma=1)
    l1, l2, l3 = structure_tensor_eigenvalues(A_elems)
    # coherence: (λ1 - λ2) / (λ1 + λ2) for the two largest
    coherence_map = (l1 - l2) / (l1 + l2 + 1e-8)

    # Hessian
    H_elems = hessian_matrix(img_3d, sigma=1, order='rc')
    h_eigs = hessian_matrix_eigvals(H_elems)  # returns sorted eigenvalues
    h_eigs = np.stack(h_eigs, axis=-1)  # shape (Z,Y,X,3)
    # simple anisotropy ratio: |λ1| / (|λ2|+|λ3|)
    h_aniso = np.abs(h_eigs[..., 0]) / (np.abs(h_eigs[..., 1]) + np.abs(h_eigs[..., 2]) + 1e-8)

    # anisotropy maps
    coherence_vals = coherence_map[mask_3d]
    mean_coherence = np.mean(coherence_vals)
    std_coherence = np.std(coherence_vals)

    h_aniso_vals = h_aniso[mask_3d]
    mean_h_aniso = np.mean(h_aniso_vals)
    std_h_aniso = np.std(h_aniso_vals)

    # directional GLCM
    glcm_feats = compute_directional_glcm_features(img_3d, mask_3d)


    feature_list = []

    ### merge regions

    for region in props:
        volume = region.area

        # geometric features
        surf_area, verts = compute_surface_area(mask_3d)
        sphericity = (np.pi**(1/3) * (6 * volume)**(2/3)) / surf_area if surf_area > 0 else np.nan
        convex_volume = compute_convex_volume(verts)
        solidity = volume / convex_volume if convex_volume and convex_volume > 0 else np.nan
        inertia = region.inertia_tensor
        eigvals, _ = np.linalg.eigh(inertia)
        elongation = np.sqrt(eigvals.min() / eigvals.max()) if eigvals.max() > 0 else np.nan

        # intensity features
        intensities = img_3d[mask_3d]
        mean_int = np.mean(intensities)
        std_int = np.std(intensities)
        min_int = np.min(intensities)
        max_int = np.max(intensities)
        med_int = np.median(intensities)
        skew_int = skew(intensities, bias=False)
        kurt_int = kurtosis(intensities, bias=False)

        # gradient-based features
        grad_vals = grad_mag[mask_3d]
        mean_grad = np.mean(grad_vals)
        std_grad = np.std(grad_vals)
        min_grad = np.min(grad_vals)
        max_grad = np.max(grad_vals)
        med_grad = np.median(grad_vals)
        skew_grad = skew(grad_vals, bias=False)
        kurt_grad = kurtosis(grad_vals, bias=False)

        vol_portion = np.sum(mask_3d) / np.prod(mask_3d.shape)

        feature_list.append({
            "label": region.label,
            # morphology
            "volume_voxels": volume,
            "vol_portion": vol_portion,
            "surface_area": surf_area,
            "sphericity": sphericity,
            "convex_volume": convex_volume,
            "solidity": solidity,
            "elongation": elongation,
            # intensity
            "intensity_mean": mean_int,
            "intensity_std": std_int,
            "intensity_min": min_int,
            "intensity_max": max_int,
            "intensity_median": med_int,
            "intensity_skew": skew_int,
            "intensity_kurtosis": kurt_int,
            # gradient
            "grad_mean": mean_grad,
            "grad_std": std_grad,
            "grad_min": min_grad,
            "grad_max": max_grad,
            "grad_median": med_grad,
            "grad_skew": skew_grad,
            "grad_kurtosis": kurt_grad,
            # anisotropy
            "coherence_mean": mean_coherence,
            "coherence_std": std_coherence,
            "hessian_aniso_mean": mean_h_aniso,
            "hessian_aniso_std": std_h_aniso,
        })
        feature_list[0].update(glcm_feats)
    return pd.DataFrame(feature_list)


In [5]:
###simple script to predict on new data
def predict_new_data(model_path: str,
                     mask: np.ndarray, 
                     img: np.ndarray,
                     sample_name: str) -> int:
    """Predict the label for a new mask and image using the trained model."""
    mask = mask.astype(np.uint8)
    assert mask.shape == img.shape, "Mask and image must have the same shape"
    df = extract_features_with_intensity_gradient(mask, img)
    vendor = sample_name.split('-')[1]
    vendor_number = vendor_to_number.get(vendor, -1)
    df['vendor'] = vendor_number
    try:
        if os.path.exists(model_path):
            model = joblib.load(model_path)
            assert hasattr(model, 'predict'), "Loaded model does not have a predict method"
            assert model.__class__.__name__ == 'RandomForestClassifier', "Loaded model is not a RandomForestClassifier"
            # Ensure the model only uses features it was trained on
            df = df[df.columns.intersection(model.feature_names_in_)]

    except FileNotFoundError:
        print(f"Model file {model_path} not found.")
        return None
    if df.empty:
        print("No features extracted from the mask. Cannot predict.")
        return None
    
    prediction = model.predict_proba(df)

    return prediction[0]

def predict_on_dataset(model_paths: dict[str, str], dataset: list) -> list:
    """Predict labels for a dataset of (mask, img) pairs."""
    predictions = []
    for img, mask, roi, sample_name, mod_name in tqdm(dataset):
        model_path = model_paths.get(mod_name, None)
        if model_path is None:
            raise FileNotFoundError(f"No model found for modality: {mod_name}. Skipping sample: {sample_name}")
        pred = predict_new_data(model_path, mask, img, sample_name)
        if pred is not None:
            predictions.append((sample_name, pred))
    return predictions

In [ ]:
### predict on the dataset
model_t1 = 'C://Users//SCoulY//Downloads//care2025_liver_biodreamer//notebook//RF_0.9722_bal_T1.pkl'
model_ged4 = 'C://Users//SCoulY//Downloads//care2025_liver_biodreamer//notebook//RF_1.0000_bal_GED4.pkl'
pred_all = predict_on_dataset({'T1': model_t1, 'GED4': model_ged4}, mod_roi_list)
# Save predictions to a CSV file
output_file = 'C://Users//SCoulY//Downloads//care2025_liver_biodreamer//notebook//LiFS_pred_non_contrast.csv'

# Please submit a single CSV file with the following format:

# ```
# Case,Setting,Subtask1_prob_S4,Subtask2_prob_S1
# XXXX-X,Contrast,0.2,0.5
# XXXX-X,NonContrast,0.6,0.1
# ...
# ```
# * Subtask1_prob_S4: Probability of class S4 (Cirrhosis)
# * Subtask2_prob_S1: Probability of class S1 (Non-fibrotic)

# Prepare CSV rows
rows = []
for sample_name, pred in pred_all:
    # Assuming pred is the predicted class label (e.g., S1, S2, S3, S4)
    prob_S4 = pred[3]
    prob_S1 = pred[0]

    rows.append([sample_name, 'NonContrast', prob_S4, prob_S1])


# Write to CSV
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Case', 'Setting', 'Subtask1_prob_S4', 'Subtask2_prob_S1'])
    writer.writerows(rows)

  0%|          | 0/60 [00:00<?, ?it/s]C:\Users\SCoulY\AppData\Local\Temp\ipykernel_8040\1817557948.py:70: FutureWarning: use_gaussian_derivatives currently defaults to False, but will change to True in a future version. Please specify this argument explicitly to maintain the current behavior
  H_elems = hessian_matrix(img_3d, sigma=1, order='rc')
  2%|▏         | 1/60 [00:13<13:36, 13.84s/it]C:\Users\SCoulY\AppData\Local\Temp\ipykernel_8040\1817557948.py:70: FutureWarning: use_gaussian_derivatives currently defaults to False, but will change to True in a future version. Please specify this argument explicitly to maintain the current behavior
  H_elems = hessian_matrix(img_3d, sigma=1, order='rc')
  3%|▎         | 2/60 [00:21<10:01, 10.37s/it]C:\Users\SCoulY\AppData\Local\Temp\ipykernel_8040\1817557948.py:70: FutureWarning: use_gaussian_derivatives currently defaults to False, but will change to True in a future version. Please specify this argument explicitly to maintain the current be